In [1]:
import pandas as pd
import re
import snowballstemmer

In [2]:
df = pd.read_csv("data/nlp_proje_metin_siniflandirma.csv",index_col=0)
df

,Text,Label
0,turkcellmüzik biraz daha gelişsin fizy yi tama...,1
1,hayat paylaşınca güzel...türkcell...,1
2,superonline varya daha iyi,1
3,hahaha kanka turkcell bu 4 ceker :d,1
4,turkcell 3 tl hediye etse bana ne güzel olur =),1
...,...,...
2995,gördüğüm en adi şerefsiz kurumsunuz,2
2996,#turkcell seni de ben boykot edicem.,2
2997,diğer hatların çekmediği yerde çeken her hattı...,2
2998,"sanirim 3g baglanti sorunu var, surekli edge o...",2


In [3]:
df.head()

,Text,Label
0,turkcellmüzik biraz daha gelişsin fizy yi tama...,1
1,hayat paylaşınca güzel...türkcell...,1
2,superonline varya daha iyi,1
3,hahaha kanka turkcell bu 4 ceker :d,1
4,turkcell 3 tl hediye etse bana ne güzel olur =),1


In [4]:
df.tail()

,Text,Label
2995,gördüğüm en adi şerefsiz kurumsunuz,2
2996,#turkcell seni de ben boykot edicem.,2
2997,diğer hatların çekmediği yerde çeken her hattı...,2
2998,"sanirim 3g baglanti sorunu var, surekli edge o...",2
2999,besiktas metropol subeniz oldukca ukela bu tur...,2


In [28]:
df["Text"] = df["Text"].astype(str)

In [6]:
df.loc[150,"Text"]

'evde superonline var buraya bekleriz'

In [7]:
pip install snowballstemmer

Note: you may need to restart the kernel to use updated packages.


In [8]:
# emojilerin kaldırılması
def remove_emoji(value):
    bfr=re.compile("[\U00010000-\U0010ffff]",flags=re.UNICODE)
    bfr=bfr.sub(r'',value)
    return bfr

In [9]:
#linklerin kaldırılması 
def remove_link(value):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',value)

In [10]:
# numerik karakterlerin kaldırılması
def remove_numeric(value):
    bfr= [item for item in value if not item.isdigit()]
    bfr = "".join(bfr)
    return bfr 

In [11]:
def remove_hashtag(value):
    if not isinstance(value, (str, bytes)):
        raise TypeError("value parametresi bir dize veya bayt benzeri nesne olmalıdır.")
    return re.sub(r'#[^\s]+', '', value)

In [12]:
# hashtaglerin kaldırılması
def remove_hashtag(value):
    return re.sub(r'#[^\s]+','',value)

In [13]:
#noktalama işaretlerinin kaldırılması
def remove_noktalama(value):
    return re.sub(r'[^\w\s]','',value) 

In [14]:
#tek karakterli ifadelerin kaldırılması
def remove_single_character(value):
    return re.sub(r'(?:^| )\w(?:$| )','',value)

In [15]:
# kullanıcı adlarının kaldırılması
def remove_username(value):
    return re.sub('@[^\s]+','',value)

In [16]:
#kök indirgeme ve stop words işlemleri
import snowballstemmer
def stem_word(value):
    stemmer= snowballstemmer.stemmer("turkish")
    value = value.lower()
    value=stemmer.stemWords(value.split())
    stop_words= ['bence','a','acaba','altı','altmış','ama','ancak','arada','artık','asla','aslında','ayrıca','az',
                 'bana','bazen','bazı','bile','biraz','bu','bunu','bunun','çoğu','çoğunu','çok','çünkü',
                 'da','daha','de','ise','defa','diye','gibi','en','kim','mı','mi','mu','mü','bir','iki',
                 'üç','dört','beş','altı','yedi','sekiz','dokuz','on','niçin','niye','şey','siz','şu',
                'her','hiç','ve','veya','ya','yani','ne','neden']
    value= [item for item in value if not item in stop_words]
    value=' ' .join(value)
    return value

In [17]:
# ön işlem fonksiyonlarının sırayla çağırılması
def pre_processing(value):
    return [(remove_emoji(remove_link
                          (remove_numeric
                           (remove_hashtag
                            (remove_noktalama
                             (remove_single_character
                              (remove_username
                               (stem_word(word))))))))) for word in value.split()]

In [18]:
df.loc[150,"Text"]

'evde superonline var buraya bekleriz'

In [19]:
df["Text_2"] = df["Text"].apply(pre_processing)
df.head()

,Text,Label,Text_2
0,turkcellmüzik biraz daha gelişsin fizy yi tama...,1,"[turkcellmüzik, , dah, geliş, fizy, yi, tamame..."
1,hayat paylaşınca güzel...türkcell...,1,"[hayat, paylaş, güzeltürkcell]"
2,superonline varya daha iyi,1,"[superonl, varya, dah, ]"
3,hahaha kanka turkcell bu 4 ceker :d,1,"[hahah, kanka, turkcell, , , ceker, d]"
4,turkcell 3 tl hediye etse bana ne güzel olur =),1,"[turkcell, , tl, hedi, et, ba, , güzel, olur, ]"


In [20]:
df.loc[0,"Text_2"]

['turkcellmüzik',
 '',
 'dah',
 'geliş',
 'fizy',
 'yi',
 'tamame',
 'bırakıca',
 'söz']

In [21]:
#boşlukların kaldırılması
def remove_space(value):
    return[item for item in value if item.strip()]

df["Text_2"]=df["Text_2"].apply(remove_space) 

In [22]:
df.loc[0,"Text_2"]

['turkcellmüzik', 'dah', 'geliş', 'fizy', 'yi', 'tamame', 'bırakıca', 'söz']

In [23]:
df.head()

,Text,Label,Text_2
0,turkcellmüzik biraz daha gelişsin fizy yi tama...,1,"[turkcellmüzik, dah, geliş, fizy, yi, tamame, ..."
1,hayat paylaşınca güzel...türkcell...,1,"[hayat, paylaş, güzeltürkcell]"
2,superonline varya daha iyi,1,"[superonl, varya, dah]"
3,hahaha kanka turkcell bu 4 ceker :d,1,"[hahah, kanka, turkcell, ceker, d]"
4,turkcell 3 tl hediye etse bana ne güzel olur =),1,"[turkcell, tl, hedi, et, ba, güzel, olur]"


In [25]:
import on_islem
import pandas as pd

In [26]:
df = pd.read_csv("data/nlp_proje_metin_siniflandirma.csv",index_col=0)
df.head()

,Text,Label
0,turkcellmüzik biraz daha gelişsin fizy yi tama...,1
1,hayat paylaşınca güzel...türkcell...,1
2,superonline varya daha iyi,1
3,hahaha kanka turkcell bu 4 ceker :d,1
4,turkcell 3 tl hediye etse bana ne güzel olur =),1


In [29]:
df["Text_2"] = df["Text"].apply(on_islem.pre_processing)
df["Text_2"] = df["Text_2"].apply(on_islem.remove_space)
df.head()

,Text,Label,Text_2
0,turkcellmüzik biraz daha gelişsin fizy yi tama...,1,"[turkcellmüzik, dah, geliş, fizy, yi, tamame, ..."
1,hayat paylaşınca güzel...türkcell...,1,"[hayat, paylaş, güzeltürkcell]"
2,superonline varya daha iyi,1,"[superonl, varya, dah]"
3,hahaha kanka turkcell bu 4 ceker :d,1,"[hahah, kanka, turkcell, ceker]"
4,turkcell 3 tl hediye etse bana ne güzel olur =),1,"[turkcell, tl, hedi, et, ba, güzel, olur]"
